In this notebook we will discuss about Chains

## Goals

Now, let's build up to a simple chain that combines 4 [concepts](https://python.langchain.com/v0.2/docs/concepts/):

* Using [chat messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as our graph state
* Using [chat models](https://python.langchain.com/v0.2/docs/concepts/#chat-models) in graph nodes
* [Binding tools](https://python.langchain.com/v0.2/docs/concepts/#tools) to our chat model
* [Executing tool calls](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling) in graph nodes 

## Messages

Chat models can use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages), which capture different roles within a conversation. 

LangChain supports various message types, including `HumanMessage`, `AIMessage`, `SystemMessage`, and `ToolMessage`. 

These represent a message from the user, from chat model, for the chat model to instruct behavior, and from a tool call. 

Let's create a list of messages. 

Each message can be supplied with a few things:

* `content` - content of the message
* `name` - optionally, a message author 
* `response_metadata` - optionally, a dict of metadata (e.g., often populated by model provider for `AIMessages`)

In [1]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

messages = [
    AIMessage(content=f"So you said you were researching ocean mammals?", name="Model")
]
messages.append(HumanMessage(content=f"Yes, that's right.", name="Lance"))
messages.append(
    AIMessage(content=f"Great, what would you like to learn about.", name="Model")
)
messages.append(
    HumanMessage(
        content=f"I want to learn about the best place to see Orcas in the US.",
        name="Lance",
    )
)

for m in messages:
    m.pretty_print()

================================== Ai Message ==================================
Name: Model

So you said you were researching ocean mammals?
================================ Human Message =================================
Name: Lance

Yes, that's right.
================================== Ai Message ==================================
Name: Model

Great, what would you like to learn about.
================================ Human Message =================================
Name: Lance

I want to learn about the best place to see Orcas in the US.


## CHAT MODELS

In [2]:
import os, getpass


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")
result = llm.invoke(messages)
type(result)

langchain_core.messages.ai.AIMessage

In [6]:
pprint(result.content)
print("-----------------")
pprint(result.response_metadata)

('One of the best places to see orcas in the U.S. is the Pacific Northwest, '
 'especially around the San Juan Islands in Washington State. The waters '
 'surrounding these islands are a prime habitat for orcas, particularly the '
 'Southern Resident killer whales, which are frequently seen between late '
 'spring and early autumn. \n'
 '\n'
 'Whale-watching tours operate from several locations, including the town of '
 'Friday Harbor on San Juan Island. Besides the boat tours, areas such as Lime '
 'Kiln Point State Park, also known as "Whale Watch Park," offer excellent '
 'opportunities for spotting orcas from the shore. \n'
 '\n'
 'Other routes to see orcas seasonally include:\n'
 '\n'
 '1. Puget Sound: Orcas can sometimes be seen from Seattle and other locations '
 'around Puget Sound.\n'
 '\n'
 '2. The Olympic Peninsula: Coastal areas, like the Strait of Juan de Fuca, '
 'can also be good for spotting transient orcas.\n'
 '\n'
 'While these areas are popular, sightings are influe

## Tools

Tools are useful whenever you want a model to interact with external systems.

External systems (e.g., APIs) often require a particular input schema or payload, rather than natural language. 

When we bind an API, for example, as a tool we given the model awareness of the required input schema.

The model will choose to call a tool based upon the natural language input from the user. 

And, it will return an output that adheres to the tool's schema. 

[Many LLM providers support tool calling](https://python.langchain.com/v0.1/docs/integrations/chat/) and [tool calling interface](https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/) in LangChain is simple. 
 
You can simply pass any Python `function` into `ChatModel.bind_tools(function)`.

What are the different ways to call a tool binded with llm model

In [10]:
# simple way to add a tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


llm_with_tools = llm.bind_tools([multiply])

In [8]:
tool_call = llm_with_tools.invoke(
    [HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")]
)

In [9]:
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_ilfOgaYT5RKpLaMMqmX0NOg5',
  'type': 'tool_call'}]

In [11]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [12]:
llm_with_tools = llm.bind_tools([multiply])

In [13]:
tool_call = llm_with_tools.invoke(
    [HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")]
)

In [14]:
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'a': 2, 'b': 3},
  'id': 'call_xglxcsf2dRG7hWoILwQ2kZ08',
  'type': 'tool_call'}]

In [16]:
from typing import Annotated, List


@tool
def multiply_by_max(
    a: Annotated[int, "scale factor"],
    b: Annotated[List[int], "list of ints over which to take maximum"],
) -> int:
    """Multiply a by the maximum of b."""
    return a * max(b)


print(multiply_by_max.args_schema.model_json_schema())
print(multiply_by_max.name)
print(multiply_by_max.description)
print(multiply_by_max.args)

{'description': 'Multiply a by the maximum of b.', 'properties': {'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}, 'required': ['a', 'b'], 'title': 'multiply_by_max', 'type': 'object'}
multiply_by_max
Multiply a by the maximum of b.
{'a': {'description': 'scale factor', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'list of ints over which to take maximum', 'items': {'type': 'integer'}, 'title': 'B', 'type': 'array'}}


In [17]:
# Best and sophisticated way to add a tool
from pydantic import BaseModel, Field


class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


@tool("multiplication-tool", args_schema=CalculatorInput, return_direct=True)
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiplication-tool
Multiply two numbers.
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}
True


In [31]:
llm_with_tools = llm.bind_tools([multiply])
tool_call = llm_with_tools.invoke(
    [
        HumanMessage(
            content=f"What is 2 multiplied by 3",
        )
    ]
)
print(tool_call.tool_calls)

[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_UTsbzNJDOTqSnfxLkseH2dc8', 'type': 'tool_call'}]


In [39]:
# Without using decorators
from langchain_core.tools import StructuredTool


def sum(a: int, b: int) -> int:
    """Sum two numbers."""
    return a + b


async def asum(a: int, b: int) -> int:
    """Sum two numbers."""
    return a + b

class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")


calculator = StructuredTool.from_function(
    func=sum,
    name="Calculator",
    description="multiply numbers",
    args_schema=CalculatorInput,
    return_direct=True,
    coroutine=asum,
)

print(calculator.invoke({"a": 2, "b": 3}))
print(await calculator.ainvoke({"a": 2, "b": 3}))
print(calculator.name)
print(calculator.description)
print(calculator.args)

5
5
Calculator
multiply numbers
{'a': {'description': 'first number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'title': 'B', 'type': 'integer'}}


In [42]:
llm_with_tools_1 = llm.bind_tools([calculator])
tool_call = llm_with_tools_1.invoke(
    [
        HumanMessage(
            content=f"What is sum of 2 and 3",
        )
    ]
)
print(tool_call.tool_calls)

[]


only tool works with llm model

We have build one chain with two tools. One is multiplication and another is sum. We can use this chain to invoke the tools in sequence.